In [1]:
import torch
import pandas as pd
import numpy as np

In [2]:
loaded_grography = torch.load('type_tensor/mlp_31_geography.pt')
loaded_history = torch.load('type_tensor/mlp_31_history.pt')
loaded_literature = torch.load('type_tensor/mlp_31_literature.pt')
loaded_music = torch.load('type_tensor/mlp_31_music.pt')
loaded_science = torch.load('type_tensor/mlp_31_science.pt')
loaded_sport = torch.load('type_tensor/mlp_31_sport.pt')

In [3]:
loaded_grography = loaded_grography[:, -1, :]
loaded_literature = loaded_literature[:, -1, :]
loaded_history = loaded_history[:, -1, :]
loaded_music = loaded_music[:, -1, :]
loaded_science = loaded_science[:, -1, :]
loaded_sport = loaded_sport[:, -1, :]

In [4]:
loaded_sport.shape

torch.Size([30, 4096])

In [5]:
data_group = {"geography":loaded_grography,
              "literature":loaded_literature, 
              "history":loaded_history, 
              "music":loaded_music, 
              "science":loaded_science, 
              "sport":loaded_sport}

In [6]:
def data_cal(loaded_tensor):
    mean_tensor = torch.mean(loaded_tensor, dim=0)
    variance_tensor = torch.var(loaded_tensor, dim=0)
    return {"mean_tensor": mean_tensor, "variance_tensor":variance_tensor}

In [7]:
data_mean_var = {}
for key, value in data_group.items(): 
    data_mean_var[key] = data_cal(value)

In [37]:
print("Number of entries that has a low variance:")
for key in data_mean_var.keys():
    print('  '+key, end=': ')
    threshold = 0.004
    less_than_threshold = data_mean_var[key]["variance_tensor"] <= threshold
    count = less_than_threshold.sum().item()  # use .item() to get the number as a standard Python number
    data_mean_var[key]["low_var_count"] = count
    print(count)

Number of entries that has a low variance:
  geography: 449
  literature: 259
  history: 168
  music: 33
  science: 96
  sport: 215


In [52]:
result_indices = {}

var_threshold = 0.004
mean_threshold = 0.08

# Iterate over each key-value pair
for key1, value1 in data_mean_var.items():
    
    cur_mean_tensor = value1['mean_tensor']
    cur_var_tensor = value1['variance_tensor']

    # Find where the value of the second tensor <=var_threshold
    indices1 = torch.nonzero(cur_var_tensor <= var_threshold).flatten()

    # An empty set to store the indices
    indices_set = set(indices1.tolist())

    if not indices_set:
        continue

    # Iterate over each of the other objects
    for key2, value2 in data_mean_var.items():
        if key1 != key2:  # Skip the current object
            comp_mean_tensor = value2['mean_tensor']

            # Subtract the values of corresponding entries
            diff = torch.abs(cur_mean_tensor - comp_mean_tensor)

            # Get indices where the absolute value of the difference >= mean_threshold
            indices2 = torch.nonzero(diff >= mean_threshold).flatten()

            # Get the intersection of the indices
            indices_set.intersection_update(set(indices2.tolist()))

    # Store the indices that hold the criteria against every other object
    result_indices[key1] = list(indices_set)

result_indices

{'geography': [1696, 1411, 2885, 3240, 3883, 2959, 595, 1780, 3381, 182, 3998],
 'literature': [1897, 2348],
 'history': [1133],
 'music': [],
 'science': [1660],
 'sport': [2724, 3366, 3852, 3410, 2429, 511]}

In [50]:
import math
print(math.sqrt(0.004))

0.06324555320336758
